In [ ]:
from sampo.generator.base import SimpleSynthetic
from sampo.generator.environment.contractor_by_wg import get_contractor_by_wg
from sampo.scheduler.genetic.base import GeneticScheduler
from sampo.scheduler.genetic.operators import TimeAndResourcesFitness

## Set parameters and generate synthetic graph

In [ ]:
graph_size = 250
seed = 123

size_of_population = 100
number_of_generation = 20

mutate_order = 0.02
mutate_resources = 0.02

fitness_constructor = TimeAndResourcesFitness()
fitness_weights = (-1, -1)
is_multiobjective = True
optimize_resources = True

save_history_to = "./history_test.json"

In [ ]:
ss = SimpleSynthetic(seed)
wg = ss.work_graph(bottom_border=graph_size)
contractors = [get_contractor_by_wg(wg)]
print(f"Generated graph with size: {wg.vertex_count}")

## Use the genetic algorithm and save history

In [ ]:
genetic_algorithm = GeneticScheduler(
    number_of_generation=number_of_generation,
    size_of_population=size_of_population,
    
    mutate_order=mutate_order,
    mutate_resources=mutate_resources,
    
    fitness_constructor=fitness_constructor,
    fitness_weights=fitness_weights,
    is_multiobjective=is_multiobjective,
    optimize_resources=optimize_resources,
    
    seed=seed,
    save_history_to=save_history_to
)
genetic_result = genetic_algorithm.schedule(wg, contractors)

## Get summary of the evolution

In [ ]:
from sampo.scheduler.utils.fitness_history import FitnessHistorySummary

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"

In [ ]:
# Load history from file
summary = FitnessHistorySummary(save_history_to)

In [ ]:
population_means = np.array(summary.get_fitness_means())

fig = px.line(x=population_means[:, 0], y=population_means[:, 1], markers=True)
fig.update_layout(height=700, width=700)
fig.show()

In [ ]:
pareto_front_means = np.array(summary.get_fitness_means(only_pareto=True))

fig = px.line(x=pareto_front_means[:, 0], y=pareto_front_means[:, 1], markers=True)
fig.update_layout(height=700, width=700)
fig.show()

In [ ]:
pareto_ratios = summary.get_pareto_to_population_ratios()

fig = px.line(y=pareto_ratios, markers=True)
fig.update_layout(height=500, width=1000)
fig.show()

In [ ]:
population_shifts = summary.get_generation_shifts()
pareto_front_shifts = summary.get_generation_shifts(only_pareto=True)

fig = px.line(y=[population_shifts, pareto_front_shifts], markers=True)
fig.data[1].line.color = "white"
fig.update_layout(height=500, width=1000, showlegend=False)
fig.show()

In [ ]:
population_uniqueness = summary.get_uniqueness_scores()
pareto_uniqueness = summary.get_uniqueness_scores(only_pareto=True)

fig = px.line(y=[population_uniqueness, pareto_uniqueness], markers=True)
fig.data[1].line.color = "white"
fig.update_layout(height=500, width=1000, showlegend=False)
fig.show()